In [1]:
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
import pytorch3d.ops
from plyfile import PlyData, PlyElement
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from argparse import ArgumentParser, Namespace
import cv2
from tqdm import tqdm
from arguments import ModelParams, PipelineParams, ModelHiddenParams
from scene import Scene, GaussianModel
from gaussian_renderer import render, render_contrastive_feature, render_segmentation, render_mask
from segment_anything import (SamAutomaticMaskGenerator, SamPredictor, sam_model_registry)
from utils.sh_utils import SH2RGB
import imageio
from utils.segment_utils import *
from kmeans_pytorch import kmeans

%load_ext autoreload
%autoreload 2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


SyntaxError: invalid syntax (feature_gaussian_model.py, line 219)

In [ ]:
DATA_ROOT = './data/hypernerf/chickchicken'
MODEL_PATH = './output/hypernerf/chickchicken'

In [ ]:
parser = ArgumentParser(description="Render script parameters")
model = ModelParams(parser, sentinel=True)
# op = OptimizationParams(parser)
pipeline = PipelineParams(parser)
hp = ModelHiddenParams(parser)
parser.add_argument("--iteration", default=-1, type=int)
parser.add_argument('--mode', default='scene', choices=['scene', 'feature'])
parser.add_argument("--configs", type=str, default = "./arguments/hypernerf/chicken.py")
# parser.add_argument('--precomputed_mask', default=None, type=str)
args = get_combined_args(parser, MODEL_PATH, 'feature')
if args.configs:
    import mmcv
    from utils.params_utils import merge_hparams
    config = mmcv.Config.fromfile(args.configs)
    args = merge_hparams(args, config)

dataset = model.extract(args)
hyperparam = hp.extract(args)

gaussians = GaussianModel(dataset.sh_degree, args.mode, hyperparam)
scene = Scene(dataset, gaussians, load_iteration=args.iteration, mode=args.mode)

Looking for config file in ./output/hypernerf/chickchicken/feature_cfg_args
Config file found at: ./output/hypernerf/chickchicken/feature_cfg_args
mode:  scene
Loading trained model at iteration 14000
load finished


114it [00:00, 31569.43it/s]

format finished
Loading Training Cameras
Loading Test Cameras
Loading Video Cameras
Deformation Net Set aabb [ 81.62185669  25.49359322 291.8190918 ] [ -34.72369003 -174.75157166   12.67879772]
Voxel Plane: set aabb= Parameter containing:
tensor([[  81.6219,   25.4936,  291.8191],
        [ -34.7237, -174.7516,   12.6788]])


loading model from exists./output/hypernerf/chickchicken/point_cloud/iteration_14000
